In [42]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import TypedDict, Annotated
from dotenv import load_dotenv
import operator


In [43]:
load_dotenv()

True

In [44]:
model=ChatOpenAI(model='gpt-4o-mini')

In [45]:
class EvaluationSchema(BaseModel):
    feedback: str = Field(description="Detailed feedback for the essay")
    score: int = Field(description='Score out of ten ', ge=0, le=10)

In [46]:
structured_model=model.with_structured_output(EvaluationSchema)

In [47]:
class CSSState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    Clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [48]:
def evaluate_language(state: CSSState):
    prompt=f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {state['essay']}'
    output=structured_model.invoke(prompt)
    return {'language_feedback':output.feedback, 'individual_scores': [output.score]}

In [49]:
def evaluate_analysis(state: CSSState):
    prompt=f'Evaluate the depth of analysis of the following essay and provide the feedback and assign a score out of 10 \n {state['essay']}'
    output=structured_model.invoke(prompt)
    return {'analysis_feedback':output.feedback, 'individual_scores': [output.score]}

In [50]:
def evaluate_thought(state: CSSState):
    prompt=f'Evaluate the clearity of thought of the following essay and provide the feedback and assign a score out of 10 \n {state['essay']}'
    output=structured_model.invoke(prompt)
    return {'Clarity_feedback':output.feedback, 'individual_scores':[output.score]}

In [51]:
def summerized_feedback(state: CSSState):
    prompt=f'Based on the following feedbacks create a summarized feedback \n language feedback - {state['language_feedback']} \n depth of analysis feedback - {state['analysis_feedback']} \n clarity of thought feedback - {state['Clarity_feedback']} '
    output=model.invoke(prompt).content
    # Avg calculate
    avg_score=sum(state['individual_scores'])/len(state['individual_scores'])
    return {'overall_feedback':output, 'avg_score':avg_score }

In [52]:
# Define graph
graph=StateGraph(CSSState)

# Add node 
graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought', evaluate_thought)
graph.add_node('summerized_feedback', summerized_feedback)

In [53]:
# add edges
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START,'evaluate_analysis')
graph.add_edge(START,'evaluate_thought')

graph.add_edge('evaluate_language','summerized_feedback')
graph.add_edge('evaluate_analysis','summerized_feedback')
graph.add_edge('evaluate_thought','summerized_feedback')

graph.add_edge('summerized_feedback', END)

In [54]:
#compile
workflow=graph.compile()

In [55]:
essay2="""pakistan and AI Time

Now world change very fast because new tech call Artificial Intel… something (AI). India also want become big in this AI thing. If work hard, India can go top. But if no careful, India go back.

India have many good. We have smart student, many engine-ear, and good IT peoples. Big company like TCS, Infosys, Wipro already use AI. Government also do program “AI for All”. It want AI in farm, doctor place, school and transport.

In farm, AI help farmer know when to put seed, when rain come, how stop bug. In health, AI help doctor see sick early. In school, AI help student learn good. Government office use AI to find bad people and work fast.

But problem come also. First is many villager no have phone or internet. So AI not help them. Second, many people lose job because AI and machine do work. Poor people get more bad.

One more big problem is privacy. AI need big big data. Who take care? India still make data rule. If no strong rule, AI do bad.

India must all people together – govern, school, company and normal people. We teach AI and make sure AI not bad. Also talk to other country and learn from them.

If India use AI good way, we become strong, help poor and make better life. But if only rich use AI, and poor no get, then big bad thing happen.

So, in short, AI time in India have many hope and many danger. We must go right road. AI must help all people, not only some. Then India grow big and world say "good job India"."""

In [56]:
initial_state={'essay':essay2}

In [57]:
result = workflow.invoke(initial_state)
print(result)

{'essay': 'pakistan and AI Time\n\nNow world change very fast because new tech call Artificial Intel… something (AI). India also want become big in this AI thing. If work hard, India can go top. But if no careful, India go back.\n\nIndia have many good. We have smart student, many engine-ear, and good IT peoples. Big company like TCS, Infosys, Wipro already use AI. Government also do program “AI for All”. It want AI in farm, doctor place, school and transport.\n\nIn farm, AI help farmer know when to put seed, when rain come, how stop bug. In health, AI help doctor see sick early. In school, AI help student learn good. Government office use AI to find bad people and work fast.\n\nBut problem come also. First is many villager no have phone or internet. So AI not help them. Second, many people lose job because AI and machine do work. Poor people get more bad.\n\nOne more big problem is privacy. AI need big big data. Who take care? India still make data rule. If no strong rule, AI do bad.\